### Ora utilizzo la media, std, e ipca calcolata sui 30 anni_even per i due file di 15 anni


In [ ]:
import sys
import os
sys.path.append(os.path.abspath(".."))

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
import shapefile as shp
import seaborn as sns
import joblib
import warnings
warnings.filterwarnings('ignore')

import importlib
import utils.data_processing
import utils.visualization
importlib.reload(utils.data_processing)
importlib.reload(utils.visualization)

# Import the functions 
from utils.data_processing import prepare_data_matrix, apply_global_standardization, perform_incremental_pca

from utils.visualization import plot_variance, scatter_plot_2d,scatter_plot_3d,  add_country_boundaries
from utils.visualization import visualization_pca_coefficient

from utils.clusterization import kmeans, elbow_analysis, gap_statistic, silhouette_analysis, davies_bouldin_analysis

In [ ]:
# Return to the original dataset to process the 2009-2023 data
gc.collect()
ds = xr.open_dataset('../era5_2009_2023.grib', engine='cfgrib')

# 
z_500 = ds['z'].sel(isobaricInhPa=[500])
t_850 = ds['t'].sel(isobaricInhPa=[850])
z_500 = z_500.rename({'isobaricInhPa': 'pressure_z'})
t_850 = t_850.rename({'isobaricInhPa': 'pressure_t'})
ds_filtered = xr.Dataset({'z': z_500, 't': t_850})
del ds, z_500, t_850
gc.collect()

0

In [3]:
X, _ = prepare_data_matrix(ds_filtered)
global_mean = np.load("global_mean_30y.npy")
global_std  = np.load("global_std_30y.npy")

X_std = (X - global_mean) / global_std

del ds_filtered
gc.collect()



Processing z...
     → z: ('time', 'pressure_z', 'latitude', 'longitude') → (5478, 64521)
Processing t...
     → t: ('time', 'pressure_t', 'latitude', 'longitude') → (5478, 64521)

Combined matrix shape: (5478, 129042)


22

In [8]:
# Load the PCA model
ipca = joblib.load("ipca_30y.pkl")
X_reduced = ipca.transform(X_std)

# 
np.save("X_pca_2009_2023.npy", X_reduced)
print("Dati ridotti salvati in X_pca_2009_2023.npy")

Dati ridotti salvati in X_pca_2009_2023.npy


tutto il sopra è stato fatto per i due quindicenni quindi ora salvando le due matrici X_pca le unisco e ottengo quella unica dei 30 anni 

#### Matrice dei 30 anni


In [5]:

X1 = np.load("X_pca_1994_2008.npy")
X2 = np.load("X_pca_2009_2023.npy")

X_pca_30y = np.concatenate([X1, X2], axis=0)

print(f"Shape unified matrix: {X_pca_30y.shape}")  # (n_righe_totali, 20)

# 
np.save("X_pca_30y.npy", X_pca_30y)
print("Matrice PCA unita salvata in X_pca_30y.npy")

Shape unified matrix: (10957, 20)
Matrice PCA unita salvata in X_pca_30y.npy
